## Графовые нейронные сети (GNNs)

Графовые нейронные сети представляют собой класс нейронных сетей, специально разработанный для работы с графовыми структурами данных. Они могут моделировать зависимости и взаимодействия между сущностями в графе, позволяя учитывать контекст и структуру данных при выполнении задач.

В GNN каждая вершина графа обновляется на основе её соседей, что позволяет модели учить представления вершин, учитывая их окружение. Такие модели широко применяются в задачах, связанных с анализом социальных сетей, биологических сетей, рекомендательных систем и графовых знаний.

## Graph embedding
Эмбеддинги в контексте графовых нейронных сетей представляют собой векторные представления вершин графа. Они извлекаются из GNN и представляют сущности в пространстве низкой размерности. Эти векторы обладают свойством сохранять структурную информацию о графе и могут использоваться для различных задач, таких как предсказание отношений или классификация вершин.

## Тестирование и Валидация

Тестирование и валидация важны для оценки работы графовых нейронных сетей.

Тестирование позволяет оценить обобщающую способность модели на новых данных, которые не участвовали в обучении. Это важно для проверки, насколько хорошо модель обобщает свои знания на новые сценарии.

Валидация используется для подбора оптимальных гиперпараметров модели. Настройка параметров происходит на основе производительности модели на отложенной валидационной выборке.

### Установка зависимостей

Для работы с графовыми нейронными сетями будем использовать [Pykeen](https://pykeen.readthedocs.io/en/stable/). Необходимо импортировать соответствующую библиотеку. В данном шаге мы устанавливаем Pykeen. Этот шаг включает установку библиотеки Pykeen с использованием команды !pip install pykeen. Pykeen предоставляет реализацию различных моделей графовых знаний и инструменты для работы с ними.

In [1]:
# Установка Pykeen
!pip install pykeen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.3/739.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 10.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.3 MB/s eta 0:00:00
  Created wheel for rexmex: filename=rexmex-0.1.3-py3-none-any.whl size=19952 sha256=64d26a3fb6710044122237cc32560b70f24bc3d83f42bffcee41f2cbcae08163
  Stored in directory: /root/.cache/pip/wheels/13/31/9d/51fb64454e6012bc88dd72b57609646432e478ba35772736d6
Successfully built rexmex


Далее, мы импортируем библиотеку Pykeen для использования её функциональности в нашем коде.

In [2]:
import pykeen

После выполнения этого шага, мы готовы к использованию Pykeen для создания, обучения и оценки моделей графовых знаний.

## Взаимодействие с графовым датасетом и описание данных

В данной работе мы будем использовать графовый датасет [Nations](https://paperswithcode.com/dataset/nations), предоставляющий информацию о связях между стран и их политических отношений. Этот небольшой датасет хорошо подходит для погружения в задачу предсказания отношений между сущностями.


Импорт необходимого модуля Pykeen. Мы импортируем модуль datasets из Pykeen для работы с графовыми датасетами.

In [3]:
from pykeen.datasets import Kinships

INFO:pykeen.utils:Using opt_einsum


Загружаем графовый датасет Nations из Pykeen. В результате выполнения этой команды будет создан объект, представляющий графовый датасет.

In [4]:
# Загрузка графового датасета Kinships
kinships_dataset = Kinships()

Мы можем вывести описание датасета, чтобы получить информацию о количестве сущностей, отношений и других характеристиках.

In [5]:
# Просмотр описания датасета
print(kinships_dataset)

Kinships(training_path="/usr/local/lib/python3.10/dist-packages/pykeen/datasets/kinships/train.txt", testing_path="/usr/local/lib/python3.10/dist-packages/pykeen/datasets/kinships/test.txt", validation_path="/usr/local/lib/python3.10/dist-packages/pykeen/datasets/kinships/valid.txt")


#### Получение тренировочных данных
Мы можем получить тренировочные факты, представленные в виде троек сущность-отношение-сущность.
Таким образом, после выполнения этих шагов, у нас есть загруженный графовый датасет, и мы можем использовать его для обучения и тестирования моделей графовых знаний в Pykeen.

In [6]:
# Получение тренировочных фактов (троек сущность-отношение-сущность)
training_data = kinships_dataset.training.mapped_triples
print(training_data[:5])

tensor([[ 0,  0, 44],
        [ 0,  2, 51],
        [ 0,  2, 52],
        [ 0,  2, 57],
        [ 0,  2, 58]])


## Описание базовой задачи

В данном контексте базовой задачей является предсказание отношений между сущностями в графовом датасете. Каждая тройка в графе представляет отношение между двумя сущностями. Например, в социальной сети, где сущности - пользователи и страницы, отношения могут быть "пользователь подписан на страницу".

In [7]:
# Импорт необходимых модулей Pykeen
from pykeen.pipeline import pipeline


#### Выбор модели
Мы выбираем модель для решения задачи предсказания отношений. В данном случае, мы используем модель [TransE](https://paperswithcode.com/method/transe), одну из популярных моделей для этой задачи.

#### Выбор датасета
Мы указываем графовый датасет, с которым будем работать. В данном примере - kinships_dataset, который содержит данные о связях в между странами.

#### Количество эпох обучения
Задаем количество эпох обучения (итераций по всему датасету).

Проверяем наличие GPU устройства и вызываем функцию pipeline(), которая запускает процесс обучения модели, тестирования и валидации.

In [8]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [9]:
# Задаем параметры для pipeline
pipeline_results = pipeline(
    model="TransE",      # Выбираем модель (TransE - одна из моделей для предсказания отношений)
    dataset=kinships_dataset,     # Выбираем графовый датасет (Nations)
    training_kwargs=dict(num_epochs=20),   # Количество эпох обучения
    device=device
)

INFO:pykeen.pipeline.api:Using device: cuda


Training epochs on cuda:0:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/34 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/34 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/34 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/34 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/34 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=2048.


Evaluating on cuda:0:   0%|          | 0.00/1.07k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.13s seconds



Этот код инициирует обучение модели на выбранном датасете, используя выбранную архитектуру (TransE) и указанные параметры. После завершения обучения, результаты выводятся, и можно провести анализ производительности модели.


Печатаем результаты, которые могут включать в себя метрики производительности, потери на тренировочных данных, а также результаты на тестовом и валидационном наборах данных.

In [10]:
# Выводим результаты
print(pipeline_results)

PipelineResult(random_seed=1865562792, model=TransE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): TransEInteraction()
  (entity_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(104, 50)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(25, 50)
    )
  )
  (weight_regularizers): ModuleList()
), training=TriplesFactory(num_entities=104, num_relations=25, create_inverse_triples=False, num_triples=8544, path="/usr/local/lib/python3.10/dist-packages/pykeen/datasets/kinships/train.txt"), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x7cb569a10910>, losses=[1.1596675374928642, 1.112264542018666, 1.1203618119744694, 1.0807912875624264, 1.072231229613809, 1.0571168380625107, 1.0308838136055891, 1.0161637856679804, 0.9925509849015404, 0.9748438403886908, 0.9642734404872445, 0.9451873986160054, 0.9441329135614283, 0.9079137707457823, 0.9128161581123576, 0.8

### Triple Scoring

Импортируем [predict_triples](https://pykeen.readthedocs.io/en/stable/api/pykeen.predict.predict_triples.html)

In [11]:
from pykeen.predict import predict_triples

Рассчитаем оценки для всех validation троек из набора данных, на котором мы обучали модель.

In [12]:
pack = predict_triples(model=pipeline_results.model, triples=kinships_dataset.validation)
df = pack.process(factory=pipeline_results.training).df



```
# This is formatted as code
```

Показаны тройки, получившие наивысший score

In [13]:
df.nlargest(n=5, columns="score")

,head_id,head_label,relation_id,relation_label,tail_id,tail_label,score
694,66,person65,20,term5,18,person21,-7.648538
705,68,person67,7,term15,39,person40,-7.687210
609,58,person58,7,term15,96,person92,-7.697379
238,22,person25,8,term16,86,person83,-7.716250
217,20,person23,10,term18,75,person73,-7.776909


В рамках самостоятельной работы попробуйте обучить и протестировать модель на любом другом датасете доступном на https://pykeen.readthedocs.io/en/stable/reference/datasets.html.